In [1]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import torch

# Load processor và model pretrained từ Hugging Face
model_name = "../model/kaggle/working/phoneme_recognition"  
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name)

# Kiểm tra thiết bị (GPU/CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('device', device)
model.to(device)

c:\Users\ADMIN\Desktop\DATN\SourceCode\train_model\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device cpu


Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (1-4): 4 x Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projec

In [4]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Tổng số tham số: {total_params:,}")


Tổng số tham số: 315,483,820


In [13]:
import soundfile as sf
import pandas as pd

# Đọc file âm thanh (16kHz, mono)
audio_path = "../temp/test2.wav"
speech, sample_rate = sf.read(audio_path)

import librosa

if sample_rate != 16000:
    speech = librosa.resample(speech, orig_sr=sample_rate, target_sr=16000)
    sample_rate = 16000

# Chuyển đổi âm thanh thành input features
inputs = processor(speech, sampling_rate=sample_rate, return_tensors="pt", padding=True)
print(inputs['input_values'])
print(pd.DataFrame(inputs))
input_values = inputs.input_values.to(device)

tensor([[-0.1086, -0.1086, -0.1086,  ..., -0.1086, -0.1086, -0.1086]])
                0
0    input_values
1  attention_mask


In [20]:
# Forward pass
with torch.no_grad():
    logits = model(input_values).logits

# Decode kết quả (từ âm thanh -> văn bản)
pad_token_id = processor.tokenizer.pad_token_id
predicted_ids = torch.argmax(logits, dim=-1)
predicted_text = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

print("Predicted Text:", predicted_text)

Predicted Text: h#[PAD]ay[PAD]l[PAD]ah[PAD]v y[PAD]ih[PAD]v[PAD]h#
